In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
from hyperopt import STATUS_OK, Trials, hp, space_eval, tpe, fmin
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, train_test_split
import seaborn as sns
import warnings
from sklearn.linear_model import LogisticRegression
import numpy as np


In [5]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

In [6]:
data_dir = Path('input')
feature_dir = Path('feature')
val_dir = Path('val')
tst_dir = Path('tst')
sub_dir = Path('sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'class'
n_fold = 5
n_class = 3
seed = 42

In [7]:
algo_name = 'lgbcv'
feature_name = 'stacking1'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'
p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'
sub_file = sub_dir / f'{model_name}.csv'

In [10]:
model_names = ['lrcv_polyfeature', 'RF_hyperopt_feature', 'lgbcv_feature','NN_hyperopt_feature']
trn = []
tst = []
feature_names = []
for model in model_names:
    trn.append(np.loadtxt(val_dir / f'{model}.val.csv', delimiter=','))
    tst.append(np.loadtxt(tst_dir / f'{model}.tst.csv', delimiter=','))
    feature_names += [f'{model}_class0', f'{model}_class1', f'{model}_class2']
    
trn = np.hstack(trn)
tst = np.hstack(tst)
feature_names

['lrcv_polyfeature_class0',
 'lrcv_polyfeature_class1',
 'lrcv_polyfeature_class2',
 'RF_hyperopt_feature_class0',
 'RF_hyperopt_feature_class1',
 'RF_hyperopt_feature_class2',
 'lgbcv_feature_class0',
 'lgbcv_feature_class1',
 'lgbcv_feature_class2',
 'NN_hyperopt_feature_class0',
 'NN_hyperopt_feature_class1',
 'NN_hyperopt_feature_class2']

In [11]:
y = pd.read_csv(trn_file, index_col=0, usecols=['id', target_col]).values.flatten()

In [12]:
X_trn, X_val, y_trn, y_val = train_test_split(trn, y, test_size=.2, random_state=seed)

In [32]:
params = {
    "objective": "multiclass",
    "n_estimators": 1000,
    "subsample_freq": 1,
    "random_state": seed,
    "n_jobs": -1,
    'bagging_fraction': 0.5
}

space = {
    "learning_rate": hp.loguniform("learning_rate", np.log(0.01), np.log(0.3)),
    "num_leaves": hp.choice("num_leaves", [15,31,63,127]),
    "colsample_bytree": hp.quniform("colsample_bytree", .5, .9, 0.1),
    "subsample": hp.quniform("subsample", .5, .9, 0.1),
    "min_child_samples": hp.choice('min_child_samples', np.arange(50,100,20))
} 

In [33]:
def objective(hyperparams):
    model = lgb.LGBMClassifier(**params, **hyperparams)
    model.fit(X=X_trn, y=y_trn,
              eval_set=[(X_val, y_val)],
              eval_metric="multi_logloss",
              early_stopping_rounds=10,
              verbose=False)
    score = model.best_score_["valid_0"]["multi_logloss"]

    return {'loss': score, 'status': STATUS_OK, 'model': model}

trials = Trials()
best = fmin(fn=objective, space=space, trials=trials,
            algo=tpe.suggest, max_evals=10, verbose=1)

hyperparams = space_eval(space, best)
n_best = trials.best_trial['result']['model'].best_iteration_
params.update(hyperparams)
print(params)

100%|██████████| 10/10 [04:02<00:00, 24.27s/trial, best loss: 0.1567761352279934]
{'objective': 'multiclass', 'n_estimators': 1000, 'subsample_freq': 1, 'random_state': 42, 'n_jobs': -1, 'bagging_fraction': 0.5, 'colsample_bytree': 0.8, 'learning_rate': 0.013114188498150362, 'min_child_samples': 50, 'num_leaves': 15, 'subsample': 0.7000000000000001}


In [34]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [37]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(**params)
    clf.fit(trn[i_trn], y[i_trn],
            eval_set=[(trn[i_val], y[i_val])],
            eval_metric='multiclass',
            early_stopping_rounds=10)
    
    p_val[i_val, :] = clf.predict_proba(trn[i_val])
    p_tst += clf.predict_proba(tst) / n_fold

training model for CV #1
[1]	valid_0's multi_logloss: 0.970335
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_logloss: 0.955118
[3]	valid_0's multi_logloss: 0.940331
[4]	valid_0's multi_logloss: 0.925967
[5]	valid_0's multi_logloss: 0.911977
[6]	valid_0's multi_logloss: 0.898325
[7]	valid_0's multi_logloss: 0.885026
[8]	valid_0's multi_logloss: 0.872059
[9]	valid_0's multi_logloss: 0.859451
[10]	valid_0's multi_logloss: 0.847129
[11]	valid_0's multi_logloss: 0.8351
[12]	valid_0's multi_logloss: 0.823359
[13]	valid_0's multi_logloss: 0.811894
[14]	valid_0's multi_logloss: 0.800705
[15]	valid_0's multi_logloss: 0.789759
[16]	valid_0's multi_logloss: 0.779045
[17]	valid_0's multi_logloss: 0.768578
[18]	valid_0's multi_logloss: 0.758343
[19]	valid_0's multi_logloss: 0.748328
[20]	valid_0's multi_logloss: 0.738544
[21]	valid_0's multi_logloss: 0.72895
[22]	valid_0's multi_logloss: 0.719564
[23]	valid_0's multi_logloss: 0.710377
[24]	valid_0's multi_logloss:

[208]	valid_0's multi_logloss: 0.200843
[209]	valid_0's multi_logloss: 0.200302
[210]	valid_0's multi_logloss: 0.199774
[211]	valid_0's multi_logloss: 0.19925
[212]	valid_0's multi_logloss: 0.198735
[213]	valid_0's multi_logloss: 0.198226
[214]	valid_0's multi_logloss: 0.19772
[215]	valid_0's multi_logloss: 0.197223
[216]	valid_0's multi_logloss: 0.196736
[217]	valid_0's multi_logloss: 0.196255
[218]	valid_0's multi_logloss: 0.195775
[219]	valid_0's multi_logloss: 0.195303
[220]	valid_0's multi_logloss: 0.19483
[221]	valid_0's multi_logloss: 0.194369
[222]	valid_0's multi_logloss: 0.193913
[223]	valid_0's multi_logloss: 0.193462
[224]	valid_0's multi_logloss: 0.193014
[225]	valid_0's multi_logloss: 0.192573
[226]	valid_0's multi_logloss: 0.192139
[227]	valid_0's multi_logloss: 0.191715
[228]	valid_0's multi_logloss: 0.191296
[229]	valid_0's multi_logloss: 0.190881
[230]	valid_0's multi_logloss: 0.190468
[231]	valid_0's multi_logloss: 0.190061
[232]	valid_0's multi_logloss: 0.189663
[23

[416]	valid_0's multi_logloss: 0.159949
[417]	valid_0's multi_logloss: 0.1599
[418]	valid_0's multi_logloss: 0.15985
[419]	valid_0's multi_logloss: 0.159802
[420]	valid_0's multi_logloss: 0.159752
[421]	valid_0's multi_logloss: 0.159703
[422]	valid_0's multi_logloss: 0.159656
[423]	valid_0's multi_logloss: 0.15961
[424]	valid_0's multi_logloss: 0.159563
[425]	valid_0's multi_logloss: 0.159516
[426]	valid_0's multi_logloss: 0.159471
[427]	valid_0's multi_logloss: 0.159426
[428]	valid_0's multi_logloss: 0.159383
[429]	valid_0's multi_logloss: 0.159334
[430]	valid_0's multi_logloss: 0.159291
[431]	valid_0's multi_logloss: 0.159245
[432]	valid_0's multi_logloss: 0.159203
[433]	valid_0's multi_logloss: 0.159161
[434]	valid_0's multi_logloss: 0.15912
[435]	valid_0's multi_logloss: 0.159073
[436]	valid_0's multi_logloss: 0.159031
[437]	valid_0's multi_logloss: 0.158988
[438]	valid_0's multi_logloss: 0.158947
[439]	valid_0's multi_logloss: 0.158909
[440]	valid_0's multi_logloss: 0.158869
[441]

[624]	valid_0's multi_logloss: 0.15529
[625]	valid_0's multi_logloss: 0.155283
[626]	valid_0's multi_logloss: 0.155276
[627]	valid_0's multi_logloss: 0.155265
[628]	valid_0's multi_logloss: 0.155258
[629]	valid_0's multi_logloss: 0.15525
[630]	valid_0's multi_logloss: 0.155242
[631]	valid_0's multi_logloss: 0.155234
[632]	valid_0's multi_logloss: 0.155226
[633]	valid_0's multi_logloss: 0.155219
[634]	valid_0's multi_logloss: 0.15521
[635]	valid_0's multi_logloss: 0.155202
[636]	valid_0's multi_logloss: 0.155194
[637]	valid_0's multi_logloss: 0.155189
[638]	valid_0's multi_logloss: 0.155181
[639]	valid_0's multi_logloss: 0.155173
[640]	valid_0's multi_logloss: 0.155166
[641]	valid_0's multi_logloss: 0.155161
[642]	valid_0's multi_logloss: 0.155153
[643]	valid_0's multi_logloss: 0.155146
[644]	valid_0's multi_logloss: 0.155139
[645]	valid_0's multi_logloss: 0.155133
[646]	valid_0's multi_logloss: 0.155125
[647]	valid_0's multi_logloss: 0.155118
[648]	valid_0's multi_logloss: 0.15511
[649

[834]	valid_0's multi_logloss: 0.15454
[835]	valid_0's multi_logloss: 0.154537
[836]	valid_0's multi_logloss: 0.154535
[837]	valid_0's multi_logloss: 0.154534
[838]	valid_0's multi_logloss: 0.154534
[839]	valid_0's multi_logloss: 0.154533
[840]	valid_0's multi_logloss: 0.154532
[841]	valid_0's multi_logloss: 0.154532
[842]	valid_0's multi_logloss: 0.154531
[843]	valid_0's multi_logloss: 0.154528
[844]	valid_0's multi_logloss: 0.154528
[845]	valid_0's multi_logloss: 0.154526
[846]	valid_0's multi_logloss: 0.154527
[847]	valid_0's multi_logloss: 0.154525
[848]	valid_0's multi_logloss: 0.154527
[849]	valid_0's multi_logloss: 0.154525
[850]	valid_0's multi_logloss: 0.154526
[851]	valid_0's multi_logloss: 0.154522
[852]	valid_0's multi_logloss: 0.154523
[853]	valid_0's multi_logloss: 0.154524
[854]	valid_0's multi_logloss: 0.154519
[855]	valid_0's multi_logloss: 0.154519
[856]	valid_0's multi_logloss: 0.154518
[857]	valid_0's multi_logloss: 0.154515
[858]	valid_0's multi_logloss: 0.154516
[

...

[143]	valid_0's multi_logloss: 0.25671
[144]	valid_0's multi_logloss: 0.255415
[145]	valid_0's multi_logloss: 0.254137
[146]	valid_0's multi_logloss: 0.252876
[147]	valid_0's multi_logloss: 0.251634
[148]	valid_0's multi_logloss: 0.250409
[149]	valid_0's multi_logloss: 0.2492
[150]	valid_0's multi_logloss: 0.248007
[151]	valid_0's multi_logloss: 0.246834
[152]	valid_0's multi_logloss: 0.245674
[153]	valid_0's multi_logloss: 0.244533
[154]	valid_0's multi_logloss: 0.243407
[155]	valid_0's multi_logloss: 0.242299
[156]	valid_0's multi_logloss: 0.241202
[157]	valid_0's multi_logloss: 0.24012
[158]	valid_0's multi_logloss: 0.239052
[159]	valid_0's multi_logloss: 0.238001
[160]	valid_0's multi_logloss: 0.236964
[161]	valid_0's multi_logloss: 0.23594
[162]	valid_0's multi_logloss: 0.23493
[163]	valid_0's multi_logloss: 0.233932
[164]	valid_0's multi_logloss: 0.232953
[165]	valid_0's multi_logloss: 0.231984
[166]	valid_0's multi_logloss: 0.23103
[167]	valid_0's multi_logloss: 0.230082
[168]	v

[351]	valid_0's multi_logloss: 0.164414
[352]	valid_0's multi_logloss: 0.164312
[353]	valid_0's multi_logloss: 0.164213
[354]	valid_0's multi_logloss: 0.164119
[355]	valid_0's multi_logloss: 0.164028
[356]	valid_0's multi_logloss: 0.163936
[357]	valid_0's multi_logloss: 0.163841
[358]	valid_0's multi_logloss: 0.163749
[359]	valid_0's multi_logloss: 0.163658
[360]	valid_0's multi_logloss: 0.16357
[361]	valid_0's multi_logloss: 0.163482
[362]	valid_0's multi_logloss: 0.163392
[363]	valid_0's multi_logloss: 0.163306
[364]	valid_0's multi_logloss: 0.163222
[365]	valid_0's multi_logloss: 0.163135
[366]	valid_0's multi_logloss: 0.163052
[367]	valid_0's multi_logloss: 0.162969
[368]	valid_0's multi_logloss: 0.16289
[369]	valid_0's multi_logloss: 0.162806
[370]	valid_0's multi_logloss: 0.162725
[371]	valid_0's multi_logloss: 0.162646
[372]	valid_0's multi_logloss: 0.162566
[373]	valid_0's multi_logloss: 0.162486
[374]	valid_0's multi_logloss: 0.162411
[375]	valid_0's multi_logloss: 0.162335
[3

[560]	valid_0's multi_logloss: 0.155801
[561]	valid_0's multi_logloss: 0.155791
[562]	valid_0's multi_logloss: 0.155778
[563]	valid_0's multi_logloss: 0.155765
[564]	valid_0's multi_logloss: 0.155751
[565]	valid_0's multi_logloss: 0.155738
[566]	valid_0's multi_logloss: 0.155725
[567]	valid_0's multi_logloss: 0.15571
[568]	valid_0's multi_logloss: 0.155697
[569]	valid_0's multi_logloss: 0.155686
[570]	valid_0's multi_logloss: 0.155673
[571]	valid_0's multi_logloss: 0.15566
[572]	valid_0's multi_logloss: 0.155648
[573]	valid_0's multi_logloss: 0.155634
[574]	valid_0's multi_logloss: 0.155623
[575]	valid_0's multi_logloss: 0.15561
[576]	valid_0's multi_logloss: 0.155599
[577]	valid_0's multi_logloss: 0.155587
[578]	valid_0's multi_logloss: 0.155574
[579]	valid_0's multi_logloss: 0.155562
[580]	valid_0's multi_logloss: 0.15555
[581]	valid_0's multi_logloss: 0.155539
[582]	valid_0's multi_logloss: 0.155524
[583]	valid_0's multi_logloss: 0.155514
[584]	valid_0's multi_logloss: 0.155506
[585

[767]	valid_0's multi_logloss: 0.154379
[768]	valid_0's multi_logloss: 0.154376
[769]	valid_0's multi_logloss: 0.154372
[770]	valid_0's multi_logloss: 0.154372
[771]	valid_0's multi_logloss: 0.154369
[772]	valid_0's multi_logloss: 0.154369
[773]	valid_0's multi_logloss: 0.154368
[774]	valid_0's multi_logloss: 0.154364
[775]	valid_0's multi_logloss: 0.154362
[776]	valid_0's multi_logloss: 0.154362
[777]	valid_0's multi_logloss: 0.154359
[778]	valid_0's multi_logloss: 0.154355
[779]	valid_0's multi_logloss: 0.154348
[780]	valid_0's multi_logloss: 0.154346
[781]	valid_0's multi_logloss: 0.154345
[782]	valid_0's multi_logloss: 0.154345
[783]	valid_0's multi_logloss: 0.154341
[784]	valid_0's multi_logloss: 0.154336
[785]	valid_0's multi_logloss: 0.154332
[786]	valid_0's multi_logloss: 0.154327
[787]	valid_0's multi_logloss: 0.154324
[788]	valid_0's multi_logloss: 0.154318
[789]	valid_0's multi_logloss: 0.154314
[790]	valid_0's multi_logloss: 0.154313
[791]	valid_0's multi_logloss: 0.154309


[973]	valid_0's multi_logloss: 0.154043
[974]	valid_0's multi_logloss: 0.154042
[975]	valid_0's multi_logloss: 0.154039
[976]	valid_0's multi_logloss: 0.154036
[977]	valid_0's multi_logloss: 0.154036
[978]	valid_0's multi_logloss: 0.154032
[979]	valid_0's multi_logloss: 0.154032
[980]	valid_0's multi_logloss: 0.154032
[981]	valid_0's multi_logloss: 0.15403
[982]	valid_0's multi_logloss: 0.15403
[983]	valid_0's multi_logloss: 0.154032
[984]	valid_0's multi_logloss: 0.154032
[985]	valid_0's multi_logloss: 0.154032
[986]	valid_0's multi_logloss: 0.15403
[987]	valid_0's multi_logloss: 0.154029
[988]	valid_0's multi_logloss: 0.154029
[989]	valid_0's multi_logloss: 0.154029
[990]	valid_0's multi_logloss: 0.154029
[991]	valid_0's multi_logloss: 0.154027
[992]	valid_0's multi_logloss: 0.154026
[993]	valid_0's multi_logloss: 0.154027
[994]	valid_0's multi_logloss: 0.154025
[995]	valid_0's multi_logloss: 0.154026
[996]	valid_0's multi_logloss: 0.154023
[997]	valid_0's multi_logloss: 0.154025
[99

In [38]:

print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

93.5050%


In [27]:
np.savetxt(p_val_file, p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

In [28]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

(80000, 1)


,class
id,
320000,0
320001,0
320002,0
320003,0
320004,0


In [29]:

sub[target_col] = np.argmax(p_tst, axis=1)
sub.head()

,class
id,
320000,2
320001,0
320002,2
320003,0
320004,2


In [30]:
sub[target_col].value_counts()

2    41000
0    29988
1     9012
Name: class, dtype: int64

In [31]:
sub.to_csv(sub_file)